In [ ]:
from pymongo import MongoClient

client = MongoClient()
db = client.regulationsgov_test # Testing DB
comments_collection = db.comments
details_collection = db.details

In [ ]:
from copy import deepcopy
import time
from CommentScraper import exists, insert

## Custom getAllComments breaks after 2 pages
def getAllComments(apibasereq, collection):
    pageNum = 1
    metaPageNum = 1
    while True: 
        apireq = deepcopy(apibasereq)
        try:
            documents = apireq.sort("lastModifiedDate").page(pageNum).get()
            print(f"[{metaPageNum}](pg {pageNum}/20) ratelimit={apireq.ratelimit}", end="")
            print(" "*100, end="\r")
        except RuntimeError:
            print("Rate Limit exceeded, retrying in 1 minute")
            time.sleep(60)
            continue
        except ConnectionError as e:
            print(e)
            continue

        if len(documents["data"]) == 0:
            break

        for doc in documents["data"]:
            if exists(doc, collection):
                continue
            insert(doc, collection)

        if pageNum >= 2: ## TESTING ONLY
            break        #
        
        if documents["meta"]["hasNextPage"] == False:
            date = documents["data"][-1]["attributes"]["lastModifiedDate"]
            apireq = apireq.lastmodified(date)
            pageNum = 1
            metaPageNum += 1
        else:
            pageNum += 1

In [ ]:
from RegAPI import RegAPI
from CommentScraper import getCommentDetails, APICommentDetailScraper

api = RegAPI(250)
getAllComments(api.endpoint("/comments"), comments_collection)
comments = [comment for comment in comments_collection.find()]
getCommentDetails(APICommentDetailScraper(api), comments, details_collection)